# GraphQl学习指南

[图书链接](https://book.douban.com/subject/34447808/)

## 基本类型

1. int
2. float
3. string
4. boolean
5. id



## 查询

### 基础查询

使用[这个接口](http://snowtooth.moonhighway.com/)

```
query {
  Trail(id:"dance-fight") {
    groomed
    accessedByLifts {
      name
      capacity
    }
  }
}
```

### 片段

DRY，复用片段fragment

使用[这个接口](http://snowtooth.moonhighway.com/)

```
fragment liftInfo on Lift {
  name
  status
  capacity
  night
  elevationGain
}
```

然后查询时可以使用这个片段

```
query {
  Lift(id: "jazz-cat") {
    ...liftInfo
    trailAccess {
      name
      difficulty
    }
  }
}
```

### 联合类型

```
query schedule {
  agenda {
    ...on Workout {
      name
      repos
    }
    ...on StudyGroup {
      name
      subject
      students
    }
  }
}
```

inline fragment

### 接口

Interface

```
query schedule {
  agenda {
    name
    start
    end
    ...on Workout {
      repos
    }
  }
}
```

当实现ScheduleItem接口的是Workout时，额外请求repos字段

### 变更

```
mutation createSong {
  addSong(title:"Taking Flower", numberOne: true, performerName: "RLC") {
    id
    title
    numberOne
  }
}
```

### 查询变量

```
mutation createSong($title:String!, $numberOne:Int, $by:String!) {
  addSong(title:$title, numberOne:$numberOne, performerName:$by) {
    id
    title
    numberOne
  }
}
```

### 订阅

```
subscription {
  liftStatusChange {
    name
    capacity
    status
  }
}
```

### 自省

inrospection

列出所有的类型

```
query {
  __schema {
    types {
      name
      description
    }
  }
}
```

获取类型的信息

```
query lifeDetails {
  __type(name:"Lift") {
    name
    fields {
      name
      description
      type {
        name
      }
    }
  }
}
```

获取根类型上所有可用的字段

```
query roots {
  __schema {
    queryType {
      ...typeFields
    }
    mutationType {
      ...typeFields
    }
    subscriptionType {
      ...typeFields
    }
  }
}

fragment typeFields on __Type {
  name
  fields {
    name
  }
}
```



## 设计Schema

### 基础定义

```
type Photo {
  id: ID!
  name: String!
  url: String!
  description: String
}
```

1. 非空字段 `!`
2. ID字段

### 标量类型

```
scalar DateTime
Type Photo {
  created: DateTime!
}
```

### 枚举

```
enum PhotoCategory {
  SELFIE
  PORTRAIT
  ACTION
  LANDSCAPE
  GRAPHIC
}
```


### Connection and List

#### 一对一连接

```
type User {
  githubLogin: ID!
  name:String
  avatar: String
}
type Photo {
  id: ID!
  name: String!
  url: String!
  description: String
  created: DateTime!
  category: PhotoCategory!
  postedBy: User!
}
```

#### 一对多连接

```
type User {
  githubLogin: ID!
  name: String
  avatar: String
  postedPhotos: [Photo!]!
}
```

#### 多对多连接

```
type User {
  ...
  inPhotos: [Photo!]!
}
type Photo {
  ...
  taggedUsers: [User!]!
}
```

Through Type

```
type User {
  friends: [Friendship!]!
}
type Friendship {
  friends: [User!]!
  howLong: Int!
  whereWeMet: Location
}
```

#### 不同类型的列表

**联合类型**

```
union AgendaItem = StudyGroup | Workout
type StudyGroup {
  name: String!
  subject: String
  students: [User!]!
}
type Workout {
  name: String!
  reps: Int!
}
type Query {
  agenda: [AgentItem!]!
}
```

**接口**

```
scalar DateTime
interface AgendaItem {
  name: String!
  start: DateTime!
  end: DateTime!
}
type StudyGroup implements AgentdaItem {
  name: String!
  start: DateTime!
  end: DateTime!
  participants: [User!]!
  topic: String!
}
type Workout implements AgendaItem {
  name: String!
  start: DateTime!
  end: DateTime!
  reps: Int!
}
type Query {
  agenda: [AgendaItem!]!
}
```

### 参数

```
type Query {
  ...
  User(githubLogin: ID!): User!
  Phto(id: ID!): Photo!
}
```

### 筛选数据

通过参数来控制返回的数据，一些应用场景：

分页

```
type Query {
  ...
  allUsers(first: Int=25 start: Int=0): [User!]!
  allPhotos(first: Int=25 start: Int=0): [Photo!]!
}
```

排序

```
emum SortDirection {
  ASCENDING
  DESENDING
}

enum SortablePhtoField {
  name
  description
  category
  created
}

type Query {
  allPhotos(sort: SortDirection = DESCENDING sortBy: SortableField = created): [Photo!]!
}
```

### 变更类型

```
type Mutation {
  postPhoto(
    name: String!
    description: String
    category: PhtoCategory=PORTAIT
  ): Photo!
}
schema {
  query: Query
  mutation: Mutation
}
```

### 输入类型

```
input PostPhotoInput {
  name: String!
  description: String
  category: PhtoCategory=PORTAIT
}
type Mutation {
  postPhoto(input: PostPhotoInput!): Photo!
}
```

### 返回类型

```
type AuthPayload {
  user: User!
  token: Stringt!
}
type Mutation {
  ...
  githubAuth(code: String!): Authpayload!
}
```

### 订阅类型

```
type Subscription {
  newPhoto: Photo!
  newUser: User!
}
schema {
  query: Query
  mutation: Mutation
  subscription: Subscription
}
```

### 文档

注释即文档